<a href="https://colab.research.google.com/github/BorisLoveDev/don-t_know-s_agents/blob/main/solo_research_best_aug_cnn_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Импорт необходимых библиотек
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
from google.colab import drive
import os
import matplotlib as plt

In [2]:
# Монтирование Google Drive
drive.mount('/content/drive')

# Путь к папке для загрузки данных
data_path = '/content/drive/My Drive/solo_research_aug_cnn/datasets'

Mounted at /content/drive


In [3]:
# Загрузка сохраненных данных
x_train_new = np.load(os.path.join(data_path, 'x_train_new.npy'))
y_train_new = np.load(os.path.join(data_path, 'y_train_new.npy'))
x_val = np.load(os.path.join(data_path, 'x_val.npy'))
y_val = np.load(os.path.join(data_path, 'y_val.npy'))
x_test = np.load(os.path.join(data_path, 'x_test.npy'))
y_test = np.load(os.path.join(data_path, 'y_test.npy'))

# Применение one-hot encoding к меткам
y_train_new_ohe = to_categorical(y_train_new, 10)
y_val_ohe = to_categorical(y_val, 10)
y_test_ohe = to_categorical(y_test, 10)

print("Data loaded and one-hot encoded successfully.")

Data loaded and one-hot encoded successfully.


In [4]:
# Печать форм всех загруженных и обработанных данных
print("Форма x_train_new:", x_train_new.shape)
print("Форма y_train_new:", y_train_new.shape)
print("Форма y_train_new_ohe:", y_train_new_ohe.shape)

print("Форма x_val:", x_val.shape)
print("Форма y_val:", y_val.shape)
print("Форма y_val_ohe:", y_val_ohe.shape)

print("Форма x_test:", x_test.shape)
print("Форма y_test:", y_test.shape)
print("Форма y_test_ohe:", y_test_ohe.shape)


Форма x_train_new: (50000, 28, 28, 1)
Форма y_train_new: (50000,)
Форма y_train_new_ohe: (50000, 10)
Форма x_val: (10000, 28, 28, 1)
Форма y_val: (10000,)
Форма y_val_ohe: (10000, 10)
Форма x_test: (10000, 28, 28, 1)
Форма y_test: (10000,)
Форма y_test_ohe: (10000, 10)


In [7]:
# Путь к папке для сохранения модели
model_path = '/content/drive/My Drive/solo_research_aug_cnn/models'

# Путь к файлу модели
model_file = os.path.join(model_path, 'mnist_model.keras')
# Загрузка модели
model = tf.keras.models.load_model(model_file)

In [8]:
# Оценка точности модели на тестовых данных
test_loss, test_accuracy = model.evaluate(x_test, y_test_ohe, verbose=2)
print(f"Точность модели на тестовых данных: {test_accuracy * 100:.4f}%")

313/313 - 1s - loss: 0.0119 - accuracy: 0.9960 - 1s/epoch - 4ms/step
Точность модели на тестовых данных: 99.6000%
